# **Poetry Rhyme Scheme Calculator**
### Intro to Computational Linguistics Final Project
#### By: Cade Edney

##### **Summary:**
While it may be easy to see a rhyme scheme from just reading it, we can also easily do so through code. This project utilizes the Gutenberg Corpus to get popular poems, and Carnegie Mellon University's Pronunciation Dictionary, to determine the rhyme scheme of the chosen poem. To do so:
1. The poem chosen is broken up stanza-by-stanza. 
2. Each stanza is then broken up line-by-line and we look at the stress tail in the line.
    * The stress tail is the final place with the greatest stress, then the remaining syllables after.
3. Each syllable that are the same or calculated to be a slant rhyme in the stanza are defined with the same letter.
    * As slant rhymes are somewhat subjective, it's hard to determine a black-and-white answer for if two words *may* rhyme or not. So, I have researched and found a source that helps:
    * The slant rhymes are detected by using weights provided by the paper "Dynamically Scoring Rhymes with Phonetic Features and Sequence Alignment", cited below.
4. Once the code is done determining each stanza's rhyme scheme in this manner, it determines the most used rhyme scheme using a Counter object.

Citation: <br>
*Bay, Benjamin, et al. “Dynamically Scoring Rhymes with Phonetic Features and Sequence Alignment.” 2019 IEEE 31st International Conference on Tools with Artificial Intelligence (ICTAI), IEEE,  2019, pp. 1581–85. DOI.org (Crossref), https://doi.org/10.1109/ICTAI.2019.00228.*


## Table of Contents
* [Constants and Imports](#constants-and-imports)
* [ARPABET and CMU Functions](#arpabet-and-cmu-functions)
* [Preprocessing Functions](#preprocessing-functions)
* [Poem Splitting Functions](#poem-splitting-functions)
* [Poem Data Functions](#poem-data-functions)
* [Rhyme Scheme Functions](#rhyme-scheme-functions)
* [Printing Results Functions](#printing-results-functions)
* [Print Options Functions](#print-options-functions)
* [User Input](#user-input)

## Constants and Imports
This section details the constant values and imports needed for the program. They are:
* Imports
    - **re:**&ensp;for regex matching
    - **nltk:**&ensp;for CMU Pronunciation Dictionary
    - **Counter:**&ensp;to find most used rhyme schemes
    - **requests:**&ensp;for UberDuck API calls, to get ARPABET translation
* Constants
    - **cmu:**&ensp;the local copy of CMU Pronunciation Dictionary
    - **rhyme_alphabet:**&ensp;the list of letters used for the rhyme schemes, starting at A and continuing through the English alphabet
    - **vowel_features:**&ensp;a dictionary mapping ARPABET vowels to a tuple containing their height, backness, roundness, and tenseness
    - **diphthong_split**&ensp;a dictionary mappeing ARPABET diphthongs to their respective split ARPABET vowels
    - **vowel_weights:**&ensp;a list of decimal values taken from the cited paper that represent the weights of the vowel features
    - **vowel_total:**&ensp;the sum of all the values in **vowel_weights**
    - **consonant_features:**&ensp;a dictionary mapping ARPABET consonants to a tuple containing their voice, place of articulation, and manner of articulation
    - **consonant_weights:**&ensp;a list of decimal values taken from the cited paper that represent the weights of the consonant features
    - **consonant_total:**&ensp;the sum of all the values in **consonant_weights**
    - **poem_paths:**&ensp;a list of the local paths of the poems available to the user
    - **poem_names:**&ensp;a list of the names of the poems available to the user
    - **poets:**&ensp;a list of the names of the poets for each of the above poems


In [92]:
# Imports
import re
import nltk
from collections import Counter
import requests

# Constants
cmu = nltk.corpus.cmudict.dict()
"""Carnegie Mellon Pronunciation Dictionary, contains ARPABET values for English words"""

cmu['minute'] = [['M', 'IH1', 'N', 'IH0', 'T']] # Because I wanted to use this pronunciation and not whatever cmu has

rhyme_alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
"""Letters used to dictate rhyme scheme. Functions will start at A and continue on through letters"""

vowel_features = {
    'AA': ('low', 'back', 'unround', 'tense'),
    'AE': ('low', 'front', 'unround', 'lax'),
    'AH': ('mid', 'central', 'unround', 'lax'),
    'AO': ('low', 'back', 'round', 'lax'),
    'AX': ('mid', 'central', 'unround', 'lax'),
    'EH': ('mid', 'front', 'unround', 'lax'),
    'EY': ('low', 'front', 'unround', 'lax'),
    'IH': ('high', 'front',  'unround', 'lax'),
    'IY': ('high', 'front',  'unround', 'tense'),
    'OH': ('mid', 'back', 'round', 'tense'),
    'UH': ('high', 'back',  'round', 'lax'),
    'UW': ('high', 'back',  'round', 'tense')
}
"""A dictionary containing the ARPABET vowels to their respective vowel features (height, backness, roundness, tenseness)"""

diphthong_splits = {
    'AW': ('AA', 'UH'),
    'AY': ('AA', 'IH'),
    'OW': ('OH', 'UH'),
    'OY': ('AO', 'IH')
}
"""A dictionary mapping ARPABET diphthongs to the ARPABET vowels the diphthong is made up of"""

vowel_weights = [0.921, 0.355, 0.979, 0.053]
""" A list of weights, where each number represents the weight of a different vowel feature (left-to-right: height, frontness, roundness, tenseness).\n
    These values were take from the paper "Dynamically Scoring Rhymes with Phonetic Features and Sequence Alignment".
"""

vowel_total = sum(vowel_weights)
"""The sum of the weights for the vowel features"""

consonant_features = {
    'B': ('voiced', 'bilabial', 'plosive'),
    'CH': ('voiceless', 'post-alveolar', 'affricate'),
    'D': ('voiced', 'alveolar', 'plosive'),
    'DH': ('voiced', 'interdental', 'fricative'),
    'DX': ('voiced', 'alveolar', 'flap'),
    'EL': ('voiced', 'alveolar', 'lateral liquid'),
    'EM': ('voiced', 'bilabial', 'nasal'),
    'EN': ('voiced', 'alveolar', 'nasal'),
    'F': ('voiceless', 'labiodental', 'fricative'),
    'G': ('voiced', 'velar', 'plosive'),
    'HH': ('voiceless', 'glottal', 'fricative'),
    'H': ('voiceless', 'glottal', 'fricative'),
    'JH': ('voiced', 'post-alveolar', 'affricate'),
    'K': ('voiceless', 'velar', 'plosive'),
    'L': ('voiced', 'alveolar', 'lateral liquid'),
    'M': ('voiced', 'bilabial', 'nasal'),
    'N': ('voiced', 'alveolar', 'nasal'),
    'NX': ('voiced', 'velar', 'affricate'),
    'NG': ('voiced', 'velar', 'affricate'),
    'P': ('bilabial', 'plosive', 'voiceless'),
    'Q': ('voiceless', 'glottal', 'plosive'),
    'R': ('voiced', 'alveolar', 'retroflex liquid'),
    'S': ('voiceless', 'alveolar', 'fricative'),
    'SH': ('voiceless', 'post-alveolar', 'fricative'),
    'T': ('voiceless', 'alveolar', 'plosive'),
    'TH': ('voiceless', 'interdental', 'fricative'),
    'V': ('voiced', 'labiodental', 'fricative'),
    'W': ('voiced', 'bilabial', 'glide'),
    'WH': ('voiceless', 'bilabial', 'glide'),
    'Y': ('voiced', 'palatal', 'glide'),
    'Z': ('voiced', 'alveolar', 'fricative'),
    'ZH': ('voiced', 'post-alveolar', 'fricative')
}
"""A dictionary containing the ARPABET consonants to their respective consonant features (voice, place of articulation, manner of articulation)"""

consonant_weights = [1.0, 0.0, 0.933]
""" A list of weights, where each number represents the weight of a different consonant feature (left-to-right: voice, place of articulation, manner of articulation).\n
    These values were take from the paper "Dynamically Scoring Rhymes with Phonetic Features and Sequence Alignment".
"""

consonant_total = sum(consonant_weights)
"""The sum of the weights for the consonant features"""

poem_paths = ['./poems/ColorintheWheat.txt', './poems/Excelsior.txt', './poems/Home.txt', './poems/JamieDouglas.txt', './poems/TheBrokenPinion.txt', './poems/TheEnsignBearer.txt', './poems/TheHeightoftheRidiculous.txt', './poems/TheHouseWithNobodyInIt.txt', './poems/ThePolishBoy.txt', './poems/TheRealRiches.txt']
"""List of the local file paths of the poems available to the user"""

poem_names = ['Color in the Wheat', 'Excelsior', 'Home', 'Jamie Douglas', 'The Broken Pinion', 'The Ensign Bearer', 'The Height of the Ridiculous', 'The House With Nobody In It', 'The Polish Boy', 'The Real Riches']
"""List of the names of the poems available to the user"""

poets = ['Hamlin Garland', 'Henry W. Longfellow', 'Edgar A. Guest', 'Anonymous', 'Hezekiah Butterworth', 'Anonymous', 'Oliver Wendell Holmes', 'Joyce Kilmer', 'Ann S. Stephens', 'John G. Saxe']
"""List of the authors of the poems available to the user"""


'List of the authors of the poems available to the user'

## ARPABET and CMU Functions
The following cell contains functions that directly relate to ARPABET and the CMU Pronunciation Dictionary. They are as follows:
* **is_vow(arpa):**&ensp;returns whether the given ARPABET character is a vowel or not

* **strip_vow(arpa):**&ensp;returns the ARPABET character without numbers, to get solely the vowel part

* **get_arpabet(word):**&ensp;uses UberDuck API to get the ARPABET sequence from the given word

* **detect_slant_rhyme(syll1, syll2, threshold=1):**&ensp;takes the two syllables and, if they have at least **threshold** vowel features in common, returns true indication it is a slant rhyme, false otherwise

In [93]:
def is_vow(arpa):
    """
    Function that takes in an ARPABET symbol and detects whether it is a vowel
        
            Parameters:
                arpa (str): An ARPABET character
                
            Returns:
                (bool): True if arpa is a vowel, false otherwise
    """
    
    return bool(re.match(r'[AEIOU][AEHOWXYR]', arpa))

def strip_vow(arpa):
    """
    Function that takes in an ARPABET symbol and, if it's a vowel, returns only the alphabetic part
        
            Parameters:
                arpa (str): An ARPABET character
                
            Returns:
                (str): The ARPABET character, without numbers
    """
    
    if is_vow(arpa):
        arpa = re.sub("[0-9]", "", arpa)
    return arpa

def get_arpabet(word):
    """
    Function that takes in a word calls UberDuck API to get the ARPABET sequence
        
            Parameters:
                word (str): A word
                
            Returns:
                (list(str)): The ARPABET sequence that corresponds to the word
    """
    
    url = f"https://api.uberduck.ai/g2p?text={word}" # put word into url
    headers = {"accept": "application/json"} # accept json values
    response = requests.get(url, headers=headers) # get json data
    arpabet_str = response.json()['arpabet'] # get the arpabet sequence
    arpabet_str = re.sub(r'{', '', arpabet_str) # preprocessing
    arpabet_str = re.sub(r'}', '', arpabet_str)
    cmu[word] = [arpabet_str.split(" ")] # put into local cmu dictionary
    return [arpabet_str.split(" ")]

def split_diphthong(dip):
    """
    Function that takes in a diphthong and separates it into two ARPABET vowels

    Args:
        dip (str): The diphthong to be split

    Returns:
        split_plus (list(str)): A list containing two ARPABET vowels, both with the same stress number as the original diphthong
    """
    split = diphthong_splits.get(strip_vow(dip)) # get the two vowels from the dictionary
    split_plus = []
    if split:
        for arpa in split:
            split_plus.append(arpa + dip[-1]) # add the stress number to the end of the new vowels
    return tuple(split_plus)

def detect_slant_rhyme(seq1, seq2):
    """
    Function that takes in two syllables and returns whether they are a slant rhyme or not. In summary, this function will look at each pair of characters in the two sequences. It will then
    get the features that are defined for those characters, then add a certain number (from vowel_weights or consonant_weights) based on whether the two features match. If at any point
    the similarity (the sum of the weights) reaches less than 90% of the total possible weights, false is returned, stating we don't have a slant rhyme. If each pair of characters has
    a similarity percentage of greater than 90%, the function will return true
        
            Parameters:
                seq1 (list(str)): A list of ARPABET characters
                seq2 (list(str)): Another list of ARPABET characters
                
            Returns:
                (bool): True if the features have a similarity greater than threshold, false otherwise
    """
    seq1 = preprocess_arpa_seq(seq1)
    seq2 = preprocess_arpa_seq(seq2)
    
    # If one sequence is smaller, shorten to the length of the smaller sequence, prioritizing vowels from back to front
    if len(seq1) < len(seq2):
        seq2 = seq2[-len(seq1):]
    elif len(seq2) < len(seq1):
        seq1 = seq1[-len(seq2):]
    
    # Take stress number out of vowel
    v1 = strip_vow(seq1[0])
    v2 = strip_vow(seq2[0])
    # Get features for both vowels
    features1 = vowel_features.get(v1)
    features2 = vowel_features.get(v2)
    
    # If unable to get features, return false
    if not (features1 and features2):
        return False
    
    similarity = 0
    
    # Go through each pair of features and add the respective weight if the two features are the same
    for i, (f1, f2) in enumerate(zip(features1, features2)):
        if f1 == f2:
            similarity += vowel_weights[i]
    
    # If the similarity score divided by the total score it could have had is less than 0.9 (or they are less than 90% similar), return false
    if similarity / vowel_total < 0.9:
        return False
    
    # Check if there was only one value in the sequence. If so, return True (only a vowel and more than 90% similar if the code gets here)
    if len(seq1) == 1:
        return True
    
    similarity = 0
    
    # Check each pair of ARPABET characters using the same method and criteria
    for i in range(1, len(seq1)):
        arpa1 = strip_vow(seq1[i])
        arpa2 = strip_vow(seq2[i])
        vowel1 = is_vow(arpa1)
        vowel2 = is_vow(arpa2)
        if vowel1 != vowel2:
            return False
        if vowel1:
            features1 = vowel_features[arpa1]
        else:
            features1 = consonant_features[arpa1]
        if vowel2:
            features2 = vowel_features[arpa2]
        else:
            features2 = consonant_features[arpa2] 
        for i, (f1, f2) in enumerate(zip(features1, features2)):
            if f1 == f2 and vowel1:
                similarity += vowel_weights[i]
            elif f1 == f2 and not vowel1:
                similarity += consonant_weights[i]
        if similarity / consonant_total < 0.9:
            return False
    
    return True


## Preprocessing Functions
The next cell contains functions that help preprocess text, which are:
* **preprocess_line(line):**&ensp;removes the following characters from the given line:
    - unicode&ensp;'—'
    - new line&ensp;'\n'

In [16]:
def preprocess_line(line):
    """
    Function that takes in a line and removes certain characters
        
            Parameters:
                line (str): A string
                
            Returns:
                line (str): The modified line, with the following characters removed:\n
                \t\t- unicode '—'
                \t\t- new line '\\n'
    """
    
    line = re.sub(r"^\n", "", line)
    line = re.sub(r"—", "", line)
    line = re.sub(r"-", " ", line)
    line = re.sub(r'\n', '', line)
    return line

def preprocess_arpa_seq(arpa_seq):
    """
    Function that takes in a sequence of ARPABET characters preprocesses it
        
            Parameters:
                arpa_seq (list(str)): A list of ARPABET characters
                
            Returns:
                new_arpa (list(str)): The modified sequence using these rules:\n
                \t\t- Any vowel with 'R' in it (an r-colored vowel), is turned into 'AX', 'R'
                \t\t- Any diphthong is split into two vowels, as defined in the 'diphthong_splits' dictionary
    """
    
    new_arpa = []
    for arpa in arpa_seq:
        # If the ARPABET character is an R-colored vowel, split it into a schwa and R.
        if is_vow(arpa) and 'R' in arpa:
            new_arpa.append('AX' + arpa[-1])
            new_arpa.append('R')
        # If the ARPABET character is just a vowel, split into two vowels and add to the running list if it's a diphthong, or just add it if it's not
        elif is_vow(arpa):
            split = split_diphthong(arpa)
            if split:
                for a in split:
                    new_arpa.append(a)
            else:
                new_arpa.append(arpa)
        # In any other case, add the ARPABET character
        else:
            new_arpa.append(arpa)
    return tuple(new_arpa)

## Poem Splitting Functions
This section of functions is what splits a poem into different pieces, such as stanzas and lines:
* **get_stanzas(poem_path):**&ensp;divides the given poem into stanzas, where each stanza is a list of lines

* **get_lines(poem_path):**&ensp;reads the poem from the given file path, separating the input into lines

In [94]:
def get_stanzas(poem_path):
    """
    Function that takes in the path to a poem and splits the poem into stanzas
        
            Parameters:
                poem_path (str): The local file path of a poem
                
            Returns:
                stanzas (list(list(str))): The poem as a list of lists of strings, where each string is a line, each list of
                \t\t\t\t       strings is a stanza, and the list of lists of strings is the poem itself
    """
    
    lines = get_lines(poem_path) # Get all the lines from the poem
    stanzas = []
    curr_stanza = []
    
    # Go through each line and add it to the current stanza if it's not a new-line character. Once a new-line character is found, then that must mean a new stanza. We add the current stanza
    # to the running list of stanzas, and clear the current stanza
    for line in lines:
        if line != '\n':
            curr_stanza.append(line)
        else:
            stanzas.append(curr_stanza)
            curr_stanza = []
    return stanzas
        

def get_lines(poem_path):
    """
    Function that takes in the path to a poem and splits the poem into lines
        
            Parameters:
                poem_path (str): The local file path of a poem
                
            Returns:
                (list(str)): The poem as a list of strings, where each string is a line and the list of strings is the poem 
                \t\t itself
    """
    
    with open(poem_path) as f:
        return f.readlines()

## Poem Data Functions
The following cell contains functions that get data directly from the poem, such as:
* **get_longest_line(poem):**&ensp;gets the lengths of the longest line, to be used for formatting the eventual output

* **get_last_syllable(line):**&ensp;returns the final syllable of the given line, starting at the last vowel, in ARPABET form

* **get_last_syllables(stanza):**&ensp;returns a list of the last syllables for each line in the given stanza

In [95]:
def get_longest_line(poem):
    """
    Function that takes in the stanzas of a poem and gets the length of the longest line amongst them
        
            Parameters:
                stanzas (list(list(str))): The poem as a list (stanzas) of lists (each stanza) of strings (each line)
                
            Returns:
                max (int): The length (in number of characters) of the longest line in the poem
    """
    
    max = 0
    for stanza in poem:
        for line in stanza:
            if len(line) > max:
                max = len(line)
    return max

def get_last_syllable(line):
    """
    Function that finds the last syllable of a line
            
            Parameters:
                line (str): A line of a poem
                
            Returns:
                syll (list(str)): The ARPABET sequence of the final syllable (starting at the last vowel) of the given line
    """
    
    line = preprocess_line(line)
    words = nltk.tokenize.word_tokenize(line)
    words=[word.lower() for word in words if word.isalpha()] # Filter words to be only the ones containing alphabet characters, and make them all lwoercase
    final_word = words[-1] # Get the last word
    final_pron = cmu.get(final_word) # Get that word's pronunciation
    if final_pron: 
        final_pron = cmu[final_word][0] # If there was a pronunciation available, take the first one
    else: # If not, use the previously defined API method to get the pronunciation
        final_pron = get_arpabet(final_word) 
        cmu[final_word] = final_pron
        final_pron = final_pron[0]
        
    syll = []
    # Add ARPABET characters from the back until the first vowel appears
    for ipa in reversed(final_pron):
        syll = [ipa] + syll
        if is_vow(ipa):
            return syll
    return syll

def get_stress_tail(line):
    """
    Function that finds the stress tail of a line
            
            Parameters:
                line (str): A line of a poem
                
            Returns:
                tail (list(str)): The ARPABET sequence of the stress tail of the line, which is the sequence of 
                \t\t       characters past the first and greatest stress.
    """
    
    # Pretty much identical to the get_last_syllable method, but modified for stress tails
    line = preprocess_line(line)
    words = nltk.tokenize.word_tokenize(line)
    words=[word.lower() for word in words if word.isalpha()]
    final_word = words[-1]
    final_pron = cmu.get(final_word)
    
    if final_pron:
        final_pron = cmu[final_word][0]
    else:
        final_pron = get_arpabet(final_word)
        cmu[final_word] = final_pron
        final_pron = final_pron[0]
    
    max_stress = -1
    max_place = 0
    # Find the vowel with the largest stress, and keep track of where it is in the sequence
    for (i, arpa) in enumerate(final_pron):
        if is_vow(arpa):
            stress = int(arpa[-1])
            if stress > max_stress:
                max_stress = stress
                max_place = i
    
    # Get the characters from the place of maximum stress onwards and return that sub-list
    tail = final_pron[max_place:]
    return tail

def get_last_words_stanza(stanza):
    """
    Function that gets the last words of a stanza. This function was used in the training of the model.
            
            Parameters:
                stanza (list(str)): the list of lines, or stanza
                
            Returns:
                final_words (list(str)): The last words of the stanza
    """
    
    final_words = []
    for line in stanza:
        line = preprocess_line(line)
        words = nltk.tokenize.word_tokenize(line)
        words = [word.lower() for word in words if word.isalpha()]
        final_words.append(words[-1])
    return final_words

def get_last_words_poem(poem_path):
    """
    Function that gets the last words of an entire poem in one list. This function was used in the training of the model.
            
            Parameters:
                poem_path (str): local path of the poem
                
            Returns:
                words (list(str)): The last words of the entire poem
    """
    
    poem = get_stanzas(poem_path)
    words = []
    for stanza in poem:
        words = words + get_last_words_stanza(stanza)
    return words

def get_last_stress_tails(stanza):
    """
    Function that gets all the stress tails in a stanza
            
            Parameters:
                stanza (list(str)): the list of lines, or stanza
                
            Returns:
                tails (list(tuple(str))): list of all the stress tails in the stanza
    """
    
    tails = []
    for line in stanza:
        if line != '':
            tails.append(tuple(get_stress_tail(line)))
    return tails  

def get_last_syllables(stanza):
    """
    Function that takes in a stanza of a poem and gets the last syllables of each line in the stanza
        
            Parameters:
                stanza (list(str)): A stanza, which is a list of lines (strings)
                
            Returns:
                sylls (list(list(str))): The list of ARPABET sequences of the final syllables of each line in the stanza
    """
    
    sylls = []
    for line in stanza:
        if line != '':
            sylls.append(tuple(get_last_syllable(line)))
    return sylls

## Rhyme Scheme Functions
This next cell is where the meat of the analysis is happening. Using many of the above functions, these will calculate the rhyme scheme for a stanza, and the whole poem:
* **calculate_scheme(stanza):**&ensp;given a stanza (list of lines), will return the rhyme scheme in the form of a list of letters taken from the constant **rhyme_alphabet** defined above

* **calculate_full_scheme(poem_path):**&ensp;uses the poem path provided to get the poem, and then calculate the rhyme scheme of each stanza. This function then returns that list of lists of letters

In [6]:
def calculate_scheme(stanza):
    """
    Function that calculates the rhyme scheme of the stanza using the final syllables
    
    Looking at each line in the stanza, this function matches the syllables that rhyme or slant rhyme, depending on the final
    vowel and consonants of each line.
        
            Parameters:
                stanza (list(str)): A stanza of a poem
                
            Returns:
                letters (list(str)): The rhyme scheme in terms of the letters in rhyme_alphabet. For example, if given a
                \t\t\t     limerick, the return value would be ['A', 'A', 'B', 'B', 'A']
    """
    
    sylls = get_last_syllables(stanza) # Get all last syllables
    letters = []
    i = 0
    curr_letter = rhyme_alphabet[i]
    rhymes = {}
    
    for syll in sylls:
        if syll in list(rhymes.keys()): # If the syllable is already a key in the map, add the corresponding letter
            letters.append(rhymes[syll])
        else:
            found = False
            for key in rhymes.keys():
                if detect_slant_rhyme(key, syll): # Check if the syllable is a slant rhyme for any of the currently stored syllables
                    letters.append(rhymes[key])
                    found = True
                    break
            if not found: # If not, it's a new rhyme! Add a new letter
                letters.append(curr_letter)
                rhymes[syll] = curr_letter
                i += 1
                curr_letter = rhyme_alphabet[i]
    return letters

def calculate_scheme_with_tails(stanza):
    """
    Function that calculates the rhyme scheme of the stanza using the stress tails
    
    Looking at each line in the stanza, this function matches the stress tails that rhyme or slant rhyme, depending on the final
    vowel and consonants of each line.
        
            Parameters:
                stanza (list(str)): A stanza of a poem
                
            Returns:
                letters (list(str)): The rhyme scheme in terms of the letters in rhyme_alphabet. For example, if given a
                \t\t\t     limerick, the return value would be ['A', 'A', 'B', 'B', 'A']
    """
    # This method is almost identical to calculate_scheme, but uses stress tails instead of the final syllables
    tails = get_last_stress_tails(stanza)
    letters = []
    i = 0
    curr_letter = rhyme_alphabet[i]
    rhymes = {}
    for tail in tails:
        if tail in list(rhymes.keys()):
            letters.append(rhymes[tail])
        else:
            found = False
            for key in rhymes.keys():
                if detect_slant_rhyme(key, tail):
                    letters.append(rhymes[key])
                    found = True
                    break
            if not found:
                letters.append(curr_letter)
                rhymes[tail] = curr_letter
                i += 1
                curr_letter = rhyme_alphabet[i]
    return letters

def calculate_full_scheme(poem_path):
    """
    Function that calculates the rhyme scheme of the entire poem by going through each stanza in the poem and compiling the
    rhyme scheme for each.
        
            Parameters:
                poem_path (str): The local file path of the poem
                
            Returns:
                schemes (list(list(str))): The rhyme schemes in terms of letters in rhyme_alphabet for each stanza
    """
    
    stanzas = get_stanzas(poem_path)
    schemes = []
    for stanza in stanzas:
        schemes.append(calculate_scheme_with_tails(stanza))
    return schemes

## Printing Results Functions
These functions will compile the work done by other functions above and print out the results.
* **print_poem_with_scheme(poem_path):**&ensp;given the poem path, gets the poem and calculates the rhyme scheme of the full poem. Then prints out a side-by-side display of the poem and rhyme scheme, with each line corresponding to the correct letter in the calculated rhyme scheme. It then returns a list of tuples of letters so that future functions can use it

* **print_most_common_scheme(tuple_scheme):**&ensp;recieves the rhyme scheme as a list of tuples of letters and puts it into a Counter. It will then print out the most common rhyme scheme, or multiple most common schemes if there are ties in that department

In [7]:
def print_poem_with_scheme(poem_path):
    """
    Function that prints the poem side-by-side with the rhyme scheme of each stanza and returns the full rhyme
    scheme for later use 
        
            Parameters:
                poem_path (str): The local file path of the poem
                
            Returns:
                tuple_scheme (list(tuple(str))): The rhyme schemes of the poem, translated to tuples for easy 
                \t\t\t\t\t\t     counting
    """
    
    stanzas = get_stanzas(poem_path)
    scheme = calculate_full_scheme(poem_path)
    max_line = get_longest_line(stanzas)
    for (stanza, seq) in zip(stanzas, scheme):
        spaces = "{:" + str(max_line) + "}"
        for (line, letter) in zip(stanza, seq):
            prep_line = preprocess_line(line) 
            print(spaces.format(prep_line) + "\t" + letter) # Formats the line to fit in the same number of space as the longest line
        print("")
    tuple_scheme = []
    for line in scheme:
        tuple_scheme.append(tuple(line))
    return tuple_scheme

def print_most_common_scheme(tuple_scheme):
    """
    Function that prints the most common rhyme scheme, or the most common rhyme schemes if multiple 
    stanzas have the same rhyme scheme with the same count
        
            Parameters:
                tuple_scheme (list(tuple(str))): The rhyme schemes of the poem, in the form of tuples for easy 
                \t\t\t\t\t\t     counting
    """
    
    counter = Counter(tuple_scheme)
    common_scheme_tuples = counter.most_common()
    max_count = common_scheme_tuples[0][1]
    schemes = []
    for scheme in common_scheme_tuples:
        if scheme[1] == max_count:
            schemes.append(scheme[0])
        else:
            break

    if len(schemes) > 1:
        print("The most common rhyme schemes are:")
        for scheme in schemes:
            print("\t" + ", ".join(scheme))
    else:
        print("The most common rhyme scheme is:")
        for scheme in schemes:
            print("\t" + ", ".join(scheme))

## Print Options Functions
The next cell has functions that relate to the user input, and how the user is presented with options
* **print_poem_options():**&ensp;prints the poem options and their respective authors from the above **poem_names** and **poets** lists

In [96]:
def print_poem_options():
    """
    Function that prints the options available to the user for choosing
    """
    
    print("Pick a poem from this list (enter 1 - 10):")
    for (i, (_, poem, author)) in enumerate(zip(poem_paths, poem_names, poets)):
        print(f" {i + 1}: {poem} - {author}")
    print()

## User Input
This is where the actual program runs and the user supplies input. The user is prompted with the list of poems available, in which the user then types a number to view the poem and its rhyme scheme, or 'quit' to stop the program

In [87]:
print_poem_options()
response = input("Choose a poem or type 'quit'!")
while not re.search(r"(?i)^quit$",response):
    if re.match(r'[0-9]+', response):
        tuple_scheme = print_poem_with_scheme(poem_paths[int(response) - 1])
        print_most_common_scheme(tuple_scheme)
        print()
        response = input("Choose another poem or type 'quit'!")
    else:
        response = input("Please input a number for the poem!")
    

Pick a poem from this list (enter 1 - 5):
 1: Color in the Wheat - Hamlin Garland
 2: Excelsior - Henry W. Longfellow
 3: Home - Edgar A. Guest
 4: Jamie Douglas - Anonymous
 5: The Broken Pinion - Hezekiah Butterworth
 6: The Ensign Bearer - Anonymous
 7: The Height of the Ridiculous - Oliver Wendell Holmes
 8: The House With Nobody In It - Joyce Kilmer
 9: The Polish Boy - Ann S. Stephens
 10: The Real Riches - John G. Saxe

Like liquid gold the wheat field lies,            	A
  A marvel of yellow and russet and green,        	B
That ripples and runs, that floats and flies,     	A
  With the subtle shadows, the change, the sheen, 	B
    That play in the golden hair of a girl,       	C
      A ripple of amber  a flare                  	D
    Of light sweeping after  a curl               	C
    In the hollows like swirling feet             	E
      Of fairy waltzers, the colors run           	F
      To the western sun                          	F
    Through the deeps of the ripening w